In [61]:
import pandas as pd

# Load the dataset into a DataFrame
dfCountries = pd.read_csv('Dados/countries of the world.csv')

dfCoffee = pd.read_csv('Dados/worldwide_coffee_habits.csv')


# Create a new column in dfCoffee to extract the country number
dfCoffee['Country_nr'] = dfCoffee['Country'].str.extract('(\d+)').astype(int)
#print(dfCoffee[['Country_nr', 'Country']])

# Filter dfCoffee to keep only the latest year for each country
latest_year_dfCoffee = dfCoffee.loc[dfCoffee.groupby('Country_nr')['Year'].idxmax()]
# order by country number then by year and then export it to a csv file
latest_year_dfCoffee = latest_year_dfCoffee.sort_values(by=['Country_nr', 'Year'])
latest_year_dfCoffee.to_csv('DadosTratados/latest_year_coffee.csv', index=False)

# Merge the datasets based on the index of dfCountries and the extracted country number in dfCoffee
# Note to teacher: here we assume the country numbers are unique and match the indices of dfCountries
merged_df = pd.merge(dfCountries.reset_index(), latest_year_dfCoffee, left_index=True, right_on='Country_nr', how='inner')

# Display the merged DataFrame
print(merged_df[['Country_x', 'Year', 'Coffee Consumption (kg per capita per year)']])

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('DadosTratados/merged_countries_coffee.csv', index=True)


                  Country_x  Year  Coffee Consumption (kg per capita per year)
2290               Albania   2023                                     9.534570
225                Algeria   2023                                     6.531126
1322        American Samoa   2023                                     7.872856
1216               Andorra   2023                                     3.812431
283                 Angola   2023                                     9.759405
1676              Anguilla   2023                                     8.875381
99       Antigua & Barbuda   2023                                     6.550550
1034             Argentina   2023                                     7.255146
1492               Armenia   2023                                     9.860390
1247                 Aruba   2023                                     3.463101
1077             Australia   2023                                     2.243379
120                Austria   2023                   

In [62]:
# create a new dataframe with only the countries with coffee consumption higher than 0
filtered_df = merged_df[merged_df['Coffee Consumption (kg per capita per year)'] > 0]
# Show the filtered DataFrame
print(filtered_df[['Country_x', 'Year', 'Coffee Consumption (kg per capita per year)']])


                  Country_x  Year  Coffee Consumption (kg per capita per year)
2290               Albania   2023                                     9.534570
225                Algeria   2023                                     6.531126
1322        American Samoa   2023                                     7.872856
1216               Andorra   2023                                     3.812431
283                 Angola   2023                                     9.759405
1676              Anguilla   2023                                     8.875381
99       Antigua & Barbuda   2023                                     6.550550
1034             Argentina   2023                                     7.255146
1492               Armenia   2023                                     9.860390
1247                 Aruba   2023                                     3.463101
1077             Australia   2023                                     2.243379
120                Austria   2023                   

In [65]:
# now import happiness data
dfHappiness = pd.read_csv('Dados/worldhappinessreport/2019.csv')
# Show the dataframe
print(dfHappiness)

     Overall rank         Country or region  Score  GDP per capita  \
0               1                   Finland  7.769           1.340   
1               2                   Denmark  7.600           1.383   
2               3                    Norway  7.554           1.488   
3               4                   Iceland  7.494           1.380   
4               5               Netherlands  7.488           1.396   
..            ...                       ...    ...             ...   
151           152                    Rwanda  3.334           0.359   
152           153                  Tanzania  3.231           0.476   
153           154               Afghanistan  3.203           0.350   
154           155  Central African Republic  3.083           0.026   
155           156               South Sudan  2.853           0.306   

     Social support  Healthy life expectancy  Freedom to make life choices  \
0             1.587                    0.986                         0.596   
1  

In [68]:
# Now merge the happiness data with the filtered data based on the country names
merged_df2 = pd.merge(filtered_df, dfHappiness, left_on='Country_x', right_on='Country or region', how='inner')
# print the merged dataframe
print(merged_df2)

Empty DataFrame
Columns: [index, Country_x, Region, Population, Area (sq. mi.), Pop. Density (per sq. mi.), Coastline (coast/area ratio), Net migration, Infant mortality (per 1000 births), GDP ($ per capita), Literacy (%), Phones (per 1000), Arable (%), Crops (%), Other (%), Climate, Birthrate, Deathrate, Agriculture, Industry, Service, Country_y, Year, Coffee Consumption (kg per capita per year), Average Coffee Price (USD per kg), Type of Coffee Consumed, Population (millions), Country_nr, Overall rank, Country or region, Score, GDP per capita, Social support, Healthy life expectancy, Freedom to make life choices, Generosity, Perceptions of corruption]
Index: []

[0 rows x 37 columns]


In [72]:
psdconfee_df2 = pd.read_csv('Dados/psd_coffee.csv')
psdconfee_df2_filtered = psdconfee_df2[(psdconfee_df2['Attribute_Description'] == 'Domestic Consumption') & (psdconfee_df2['Calendar_Year'] == 2020)]
psdconfee_df2_grouped = psdconfee_df2_filtered.groupby('Country_Name').sum()
print(psdconfee_df2_grouped)

              Commodity_Code                    Commodity_Description  \
Country_Name                                                            
Algeria              1422200               Coffee, GreenCoffee, Green   
Angola               2133300  Coffee, GreenCoffee, GreenCoffee, Green   
Argentina            1422200               Coffee, GreenCoffee, Green   
Australia            1422200               Coffee, GreenCoffee, Green   
Benin                 711100                            Coffee, Green   
...                      ...                                      ...   
Venezuela            1422200               Coffee, GreenCoffee, Green   
Vietnam              2133300  Coffee, GreenCoffee, GreenCoffee, Green   
Yemen                 711100                            Coffee, Green   
Zambia                711100                            Coffee, Green   
Zimbabwe              711100                            Coffee, Green   

             Country_Code  Market_Year  Calendar_Y